In [1]:
!pip install cassandra-driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 55.7 MB/s eta 0:00:00


In [2]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

cloud_config = {
    "secure_connect_bundle": "secure-connect-nyc-yellow-taxi-db.zip"  # replace <</PATH/TO/>> with the path where your downloaded bundle was downloaded (make sure to place this python file in the same place as well)
}
auth_provider = PlainTextAuthProvider(
    "yiGvbsUpoEqllXBsXXwOjZBc",
    "p,b-6bj-cMT1IR3dv,kGm6A8Q+yZg0jcMHWivO2XU0y9+evSLpYZ923mLvs,ocJ-CLeOdYI8EeiBoa7jR.r3fOFtpiQwO+Zdb2aq4FqaO4xSEXj_TF4KFkHoAs5N6mt_",
)  # replace <<CLIENT ID>> and <<CLIENT SECRET>> with the ClientID and Client Secret from your generated token
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()
session.set_keyspace(
    "nyc_yellow_taxi_db")  # replace <<KEYSPACENAME>> with the name of the keyspace you created in DataStax

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(138591776420864) c072af12-8ab0-4392-856a-67d9d9a3c4f5-us-east1.db.astra.datastax.com:29042:d7422526-b25b-4d76-8ddd-4e67a20fc75d> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


In [3]:
# -✔️- Reading listings from CSV into dataframe

import pandas as pd

# read the csv files into the dataframes
trip_data = pd.read_csv('taxi_trip_data.csv')
zone_data = pd.read_csv('taxi_zone_geo.csv')

In [4]:
# a) Remove the columns “store_and_fwd_flag”, “rate_code” and “total_amount” from taxitripdata
trip_data = trip_data.drop(['store_and_fwd_flag', 'rate_code', 'total_amount'], axis=1)


In [5]:
# b) Drop rows with missing details
trip_data = trip_data.dropna()


In [6]:
#merge the datasets
merged_df = pd.merge(trip_data, zone_data, left_on='pickup_location_id', right_on='zone_id', how='left', suffixes=('_pickup', '_dropoff'))
merged_df = pd.merge(merged_df, zone_data, left_on='dropoff_location_id', right_on='zone_id', how='left', suffixes=('_pickup', '_dropoff'))


In [7]:
merged_df = merged_df.dropna()

In [8]:
# print the first 5 rows of the dataframe
print(merged_df.head())

   vendor_id      pickup_datetime     dropoff_datetime  passenger_count  \
0          1  2018-05-11 17:40:16  2018-05-11 17:55:35                1   
1          2  2018-03-22 23:01:41  2018-03-22 23:25:36                1   
2          2  2018-07-24 09:58:45  2018-07-24 10:22:37                1   
3          2  2018-12-21 18:28:55  2018-12-21 18:35:45                1   
4          1  2018-08-15 13:58:39  2018-08-15 14:05:25                1   

   trip_distance  payment_type  fare_amount  extra  mta_tax  tip_amount  ...  \
0           1.60             1         11.5    1.0      0.5        0.00  ...   
1           9.52             1         28.5    0.5      0.5        5.96  ...   
2           2.17             1         15.5    0.0      0.5        1.50  ...   
3           0.86             2          6.0    1.0      0.5        0.00  ...   
4           0.30             2          5.5    0.0      0.5        0.00  ...   

   pickup_location_id  dropoff_location_id  zone_id_pickup  \
0     

In [9]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55701 entries, 0 to 56822
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   vendor_id            55701 non-null  int64  
 1   pickup_datetime      55701 non-null  object 
 2   dropoff_datetime     55701 non-null  object 
 3   passenger_count      55701 non-null  int64  
 4   trip_distance        55701 non-null  float64
 5   payment_type         55701 non-null  int64  
 6   fare_amount          55701 non-null  float64
 7   extra                55701 non-null  float64
 8   mta_tax              55701 non-null  float64
 9   tip_amount           55701 non-null  float64
 10  tolls_amount         55701 non-null  float64
 11  imp_surcharge        55701 non-null  float64
 12  pickup_location_id   55701 non-null  int64  
 13  dropoff_location_id  55701 non-null  float64
 14  zone_id_pickup       55701 non-null  float64
 15  zone_name_pickup     55701 non-null 

In [17]:
# Truncate the taxi_data table
truncate_table_taxi_data = "TRUNCATE trip_data;"
session.execute(truncate_table_taxi_data)



In [42]:
# session.execute("DROP TABLE trip_data;");
# session.execute("DROP TABLE taxi_data;");



In [44]:
session.execute("DROP TYPE dropoff_location_type;");


In [45]:

# create the table for trip_data, with a UDT for the pickup and dropoff location from the zone_geo table
session.execute(
    """
    CREATE TYPE pickup_location (
        zone_id FLOAT,
        zone_name TEXT,
        borough TEXT,
        zone_geom TEXT
    );
    """
)

session.execute(
    """
    CREATE TYPE dropoff_location (
        zone_id FLOAT,
        zone_name TEXT,
        borough TEXT,
        zone_geom TEXT
    );
    """
)



In [ ]:

# columns in the trip_data table:
# vendor_id, pickup_datetime, dropoff_datetime, passenger_count, trip_distance, rate_code, store_and_fwd_flag, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount,
# imp_surcharge, total_amount, pickup_location_id, dropoff_location_id, pickup_location, dropoff_location
#  zone_id_x zone_name_x  borough_x  zone_geom_x  zone_id_y zone_name_y  borough_y  zone_geom_y

# create the table for trip_data
session.execute(
    """
CREATE TABLE trip_data (
    vendor_id INT,
    pickup_datetime TIMESTAMP,
    dropoff_datetime TIMESTAMP,
    passenger_count FLOAT,
    trip_distance FLOAT,
    payment_type INT,
    fare_amount FLOAT,
    extra FLOAT,
    mta_tax FLOAT,
    tip_amount FLOAT,
    tolls_amount FLOAT,
    imp_surcharge FLOAT,
    pickup_location location,
    dropoff_location location,
    PRIMARY KEY ((pickup_datetime, dropoff_datetime), vendor_id)
);


    """
)


In [50]:
# c) Insert the data in the database as you see fit
from cassandra.cluster import Cluster
# Iterate over rows in merged_df DataFrame
# Iterate over rows in merged_df DataFrame
for index, row in merged_df.iterrows():
    # Extract data for insertion
    vendor_id = int(row['vendor_id'])
    pickup_datetime = row['pickup_datetime']
    dropoff_datetime = row['dropoff_datetime']
    passenger_count = int(row['passenger_count'])
    trip_distance = row['trip_distance']
    payment_type = row['payment_type']
    fare_amount = row['fare_amount']
    extra = row['extra']
    mta_tax = row['mta_tax']
    tip_amount = row['tip_amount']
    tolls_amount = row['tolls_amount']
    imp_surcharge = row['imp_surcharge']

    # Convert pickup_location and dropoff_location to Cassandra map type
# Convert pickup_location and dropoff_location to Cassandra map type
    pickup_location_data = {
        'zone_id': int(row['zone_id_pickup']),
        'zone_name': row['zone_name_pickup'],
        'borough': row['borough_pickup'],
        'zone_geom': row['zone_geom_pickup']
    }
    dropoff_location_data = {
        'zone_id': int(row['zone_id_dropoff']),
        'zone_name': row['zone_name_dropoff'],
        'borough': row['borough_dropoff'],
        'zone_geom': row['zone_geom_dropoff']
    }

    # Insert the data into the trip_data table
    session.execute(
        """
        INSERT INTO trip_data (vendor_id, pickup_datetime, dropoff_datetime, passenger_count, trip_distance, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, imp_surcharge, pickup_location, dropoff_location)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (vendor_id, pickup_datetime, dropoff_datetime, passenger_count, trip_distance, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, imp_surcharge, pickup_location_data, dropoff_location_data)
    )


InvalidRequest: Error from server: code=2200 [Invalid query] message="Invalid map literal for pickup_location of type location"

In [ ]:
# d) Calculate the duration for each trip and add it as a new field in your database (e7sb el duration men pickup date le dropoff date)
# add a new column to the trip_data dataframe for the duration of the trip
trip_data["duration"] = trip_data["dropoff_datetime"] - trip_data["pickup_datetime"]

# add a new column to the trip_data table for the duration of the trip
session.execute(
    """
    ALTER TABLE trip_data ADD duration timestamp
    """
)

# update the duration column in the trip_data table with the duration of the trip
for x in range(0, 25000):
    pickup_datetime = trip_data["pickup_datetime"][x]
    dropoff_datetime = trip_data["dropoff_datetime"][x]
    duration = dropoff_datetime - pickup_datetime

    session.execute(
        """
        UPDATE trip_data
        SET duration = %s
        WHERE pickup_datetime = %s AND dropoff_datetime = %s, vendor_id = %s
        """,
        (duration, pickup_datetime, dropoff_datetime, trip_data["vendor_id"][x]),
    )

In [ ]:
# e) Use “fare_amount”, “extra”, “mta_tax”, “tip_amount”, “tolls_amount” and “imp_surcharge” to calculate the total trip cost and add it as a new field in your database (e7sb el total cost)


In [ ]:
# f) What is the most common payment type used per time of day? Hint: time of day meaning morning, afternoon, evening, night
# create a new column in the trip_data dataframe for the time of day
trip_data["time_of_day"] = pd.cut(
    trip_data["pickup_datetime"].dt.hour,
    bins=[0, 6, 12, 18, 24],
    labels=["night", "morning", "afternoon", "evening"],
)

# create a new column in the trip_data table for the time of day
session.execute(
    """
    ALTER TABLE trip_data ADD time_of_day text
    """
)

# update the time_of_day column in the trip_data table with the time of day
for x in range(0, 25000):
    time_of_day = pd.cut(
        trip_data["pickup_datetime"].dt.hour,
        bins=[0, 6, 12, 18, 24],
        labels=["night", "morning", "afternoon", "evening"],
    )

    session.execute(
        """
        UPDATE trip_data
        SET time_of_day = %s
        WHERE pickup_datetime = %s AND dropoff_datetime = %s, vendor_id = %s
        """,
        (time_of_day, trip_data["pickup_datetime"][x], trip_data["dropoff_datetime"][x], trip_data["vendor_id"][x]),
    )

# query the trip_data table to find the most common payment type used per time of day
result = session.execute(
    """
    SELECT time_of_day, payment_type, COUNT(*)
    FROM trip_data
    GROUP BY time_of_day, payment_type
    """
)


In [ ]:
# g) What is the average tip amount per passenger count?
# query the trip_data table to find the average tip amount per passenger count
result = session.execute(
    """
    SELECT passenger_count, AVG(tip_amount)
    FROM trip_data
    GROUP BY passenger_count
    """
)

# print the average tip amount per passenger count
for row in result:
    print(row)

In [ ]:
# h) What are the best 5 locations for drivers to pick up passengers from? (based on the trip count)
# query the trip_data table to find the best 5 locations for drivers to pick up passengers from
result = session.execute(
    """
    SELECT pickup_location.zone_name, COUNT(*)
    FROM trip_data
    WHERE pickup_location.zone_name IS NOT NULL
    GROUP BY pickup_location.zone_name
    ORDER BY COUNT(*) DESC
    LIMIT 5
    """
)

# print the best 5 locations for drivers to pick up passengers from
for row in result:
    print(row)

In [ ]:
# Is there a correlation between trip distance and the tip amount? (not to be done using the correlation calculation)
# create a new cloumn tip percentage, and calculate the tip percentage for each trip by dividing the tip amount by the total amount, and compare it to the trip distance

In [ ]:
# Display the results of f, g and h using visualizations (will be graded based on the creativity and efficiency of the visualizations)
